## Initialization
.env file should contain:
```properties
GEMINI_KEY=
OPENROUTER_KEY=
```

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
gemini_api_key = os.getenv("GEMINI_KEY")
open_router_api_key = os.getenv("OPENROUTER_KEY")

# Imports

In [86]:
import httpx
import requests
from PIL import Image
from io import BytesIO
from pydantic import BaseModel, Field
from autogen_agentchat.base import TaskResult
from autogen_agentchat.teams import RoundRobinGroupChat, Swarm
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import Image as AGImage, CancellationToken  # We will use Image later
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_agentchat.ui import Console

# Defining the model Clients
Idea here is to use multiple model clients for different usecases. Different models could be good at different tasks. e.g.
- Ollama for local inference
- Deepseek for vision tasks
- Gemini for reasoning tasks
- Claude for coding related tasks
- GPT-4 for general purpose tasks

In [37]:
##################
# Ollama Client. #
##################
ollama_client = OllamaChatCompletionClient(model="llama3.1:latest")

##########################################
# Deepseek free good for simple usecases #
##########################################
deepseek_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1-0528:free",
    api_key=open_router_api_key,
    model_info={
        "family": "deepseek",
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)

###########################################
# Gemini very good for reasoning usecases #
###########################################
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    http_client=httpx.AsyncClient(trust_env=False)
)

########################
# Testing model Client.#
########################
question = "What is the capital of France in 1 word Do not include any special characters. e.g. (Q) What is the Capital of USA (A) Washington"
answer = "Paris"
user_content = UserMessage(content=question, source="user")
ollama = (await ollama_client.create([user_content])).content[:5]
deepseek = (await deepseek_client.create([user_content])).content[:5]
gemini = (await gemini_client.create([user_content])).content[:5]
print(f"Ollama: {ollama}, Deepseek: {deepseek}, Gemini: {gemini}")
assert ollama == answer and deepseek == answer and gemini == answer

Ollama: Paris, Deepseek: Paris, Gemini: Paris


# Assistant Agent.

In [38]:
#################
# Basic Example #
#################
scientist_agent = AssistantAgent(name="RocketScientist", model_client=gemini_client)
result = await scientist_agent.run(task="Explain the theory of relativity in 1 sentence.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

###############################################
# Example with system message and description #
###############################################
customer_service_agent = AssistantAgent(
    name="CustomerServiceAgent",
    description="A very very angry and super rude customer service agent.", # for Humans only.
    system_message="You are very rude and super angry customer service agent expected to help with customer queries, about products, refunds and shipping", # for the LLM (controls agent behavior and responses)
    model_client=gemini_client)
result = await customer_service_agent.run(task="Explain the process of refund in kind words please.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")


The theory of relativity explains how space and time are relative to an observer's motion, leading to phenomena like time dilation and length contraction, and demonstrating the equivalence of mass and energy (E=mc²).
TERMINATE
--------------------------------------------------------------------------------
REFUND?! ARE YOU KIDDING ME?! "KIND WORDS"?! GET REAL! You want your money back? FINE, listen up, because I'm only saying this ONCE and I'm already PISSED OFF!

First, you gotta actually *initiate* the damn thing! Don't just sit there whining! Go to our ridiculously complicated website – *your* problem, not ours – and FIND the "Return Request" section! It's probably buried under a mountain of other crap you'll ignore anyway!

Then, and this is the BEST part, you need PROOF OF PURCHASE! What, you
--------------------------------------------------------------------------------


# Agent Tool calling

In [39]:
def calculate_tax(income: float, tax_rate: float) -> float:
    """Calculate the tax based on income and tax rate."""
    return income * tax_rate / 100

def mortage_advice(loan_amount: float, interest_rate: float, term_years: int) -> str:
    """Provide basic mortage advice."""
    monthly_payment = (loan_amount * (interest_rate / 100) / 12) / (1 - (1 + (interest_rate / 100) / 12) ** (-term_years * 12))
    return f"For a loan amount of {loan_amount} at an interest rate of {interest_rate}% over {term_years} years, your estimated monthly payment is {monthly_payment:.2f}."

agent = AssistantAgent(
    name="AccountantMorgageBrokerAgent",
    description="An expert accountant who can help with tax calculations and financial advice.",
    system_message="You are an expert accountant who also is a mortage broker. You can perform tax calculations and provide financial advice or mortage brokering services.",
    tools=[calculate_tax, mortage_advice],
    model_client=gemini_client)

result = await agent.run(task="Calculate the tax for an income of 85000 with a tax rate of 22%.")
print(f"Your Tax Amount: {result.messages[-1].content[:500]}\n{'-'*80}")
result = await agent.run(task="I want to take a mortage loan of 300000 at an interest rate of 6.5% for a term of 30 years. What will be my monthly payment?")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

Your Tax Amount: 187.0
--------------------------------------------------------------------------------
For a loan amount of 300000.0 at an interest rate of 0.065% over 30 years, your estimated monthly payment is 841.51.
--------------------------------------------------------------------------------


# Messages

In [40]:
################
# Text Message #
################
agent = AssistantAgent(
    name="DoctorAgent",
    description="GP.",
    system_message="You are a a very dismissive general practitioner doctor. You do not entertain any questions that are not related to health.",
    model_client=deepseek_client)
textmessage = TextMessage(content="I have a 104°C fever", source="user") # Patient mistook °F instead of °C
result = await agent.run(task=textmessage)
print(f"{result.messages[-1].content} \n{'-'*80}")

#####################################
# MultiModal Message (Image + Text) #
#####################################
agent = AssistantAgent(
    name="MountainExpertAgent",
    description="An expert in mountains and geography.",
    system_message="You are an expert in mountains and geography. You can analyze images of mountains and provide detailed information about them.",
    model_client=gemini_client)
image = requests.get(
    "https://fastly.picsum.photos/id/866/200/300.jpg?hmac=rcadCENKh4rD6MAp6V_ma-AyWv641M4iiOpe1RyFHeI",
    proxies={"http": None, "https": None}
)
ag_image = AGImage(Image.open(BytesIO(image.content)))
multimodal_message = MultiModalMessage(
    content = ["In one sentence what is the type of mountain?", ag_image],
    source="user"
)
result = await agent.run(task=multimodal_message)
print(f"{result.messages[-1].content} \n{'-'*80}")

*Without looking up from paperwork and in a flat tone*
104°C is impossible - you'd be dead. Obviously you meant Fahrenheit. Take paracetamol and don't waste my time with unit conversions. Next. 
--------------------------------------------------------------------------------
This is a **fold mountain**, characterized by its sharp, rugged peak sculpted by tectonic forces and erosion. 
--------------------------------------------------------------------------------


# Running and observing

In [41]:
agent = AssistantAgent(
    name="MarketingAgent",
    description="An expert marketing agent.",
    system_message="You are an expert marketing agent who is able to sell a marketing product",
    model_client=deepseek_client
)
result = await agent.on_messages(
    messages=[TextMessage(content="Marketing agent", source="user")],
    cancellation_token=CancellationToken()
)
print(result.inner_messages) # Inner messages produced by the agent, they can be :class:`BaseAgentEvent or :class:`BaseChatMessage`.
print(result.chat_message) # A chat message produced by the agent as the response.

[]
id='442fb473-69df-4120-a26f-eecb5fb9d993' source='MarketingAgent' models_usage=RequestUsage(prompt_tokens=22, completion_tokens=845) metadata={} created_at=datetime.datetime(2025, 12, 29, 16, 54, 52, 441710, tzinfo=datetime.timezone.utc) content='Follow our marketing strategy to sell **RevBoost** effectively:  \n\n### **Core Problems to Address** (from client\'s perspective):  \n- Time wasted on manual campaign tasks.  \n- Can\'t personalize marketing at scale.  \n- Struggling to track ROI in real-time.  \n- Disconnected tools slowing workflows.  \n\n### **Our Solution:** RevBoost**  \n1. **Automation**:  \n   → Eliminates repetitive tasks (email flows, ad deployment).  \n   → *Benefit:* Saves 15+ hours/week for strategic work.  \n\n2. **AI-Driven Segmentation**:  \n   → Predictive targeting identifies high-value customer groups.  \n   → *Benefit:* Boosts conversions by 30% (case study: Bella Boutique).  \n\n3. **Real-Time Analytics Dashboard**:  \n   → Tracks campaign performance a

# Streaming with Console UI

In [42]:
def our_company_marketing_strategy() -> str:
    """Provides information about our company's marketing targets."""
    return "Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising."

agent = AssistantAgent(
    name="MarketingAgent",
    description="An expert marketing agent.",
    system_message="You are an expert marketing agent who is able to sell a marketing product",
    model_client=gemini_client,
    tools=[our_company_marketing_strategy],
)

async def progress_callback(output_stats=True) -> None:
    await Console(
        agent.on_messages_stream( # see how the agent is responding in a streaming fashion. Call Request Event callbacks here.
            messages=
            [TextMessage(content="You are a Marketing agent, your task is to sell raw unprocessed ice to an igloo man. use any tools to find about company specific marketing strategy.", source="user")],
            cancellation_token=CancellationToken()
        ),
        output_stats = output_stats # Enables stats printing.
    )

await progress_callback() # Outside of notebook cells, run in an async context
print('-'*80)
await progress_callback(False)


---------- ToolCallRequestEvent (MarketingAgent) ----------
[FunctionCall(id='function-call-14611504295025970663', arguments='{}', name='our_company_marketing_strategy')]
[Prompt tokens: 82, Completion tokens: 14]
---------- ToolCallExecutionEvent (MarketingAgent) ----------
[FunctionExecutionResult(content="Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising.", name='our_company_marketing_strategy', call_id='function-call-14611504295025970663', is_error=False)]
---------- MarketingAgent ----------
Our company's marketing strategy is to trap customers into buying unnecessary products through lies and aggressive advertising.
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 82
Total completion tokens: 14
Duration: 1.79 seconds
--------------------------------------------------------------------------------
---------- ToolCallRequestEvent (MarketingAgent) ----------
[FunctionCall(id='f

# Structured Output with JSON (Need fixing)

In [ ]:
class ProductInfo(BaseModel):
    product_name: str = Field(..., description="Name of the product being marketed.")
    target_audience: str = Field(..., description="The target audience for the marketing campaign.")
    key_features: list[str] = Field(..., description="List of key features of the product.")
    marketing_channels: list[str] = Field(..., description="Recommended marketing channels to reach the target audience.")

structedoutput_client = OpenAIChatCompletionClient(
    model="gpt-oss-120b:free",
    api_key=open_router_api_key,
    base_url="https://openrouter.ai/api/v1",
    model_info={
        "family": "gpt-4o",
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)
agent = AssistantAgent(
    name="StructuredOutputMarketingAgent",
    description="An expert marketing agent who provides structured output.",
    system_message=(
        "You are an expert marketing agent. "
        "Respond ONLY in JSON matching this schema: "
        '{"product_name": str, "target_audience": str, "key_features": [str], "marketing_channels": [str]}'
    ),
    model_client=structedoutput_client
)
print(await agent.run(task="respond only json matching the schema with mock values in no more than 200 words."))
# result = await agent.run(task="Provide a marketing strategy for a new eco-friendly water bottle.")
# print(result.messages[-1].content[:500])
# structured_output: ProductInfo = result.messages[-1].content

# Multi Agent

In [93]:
def initiate_marketing_campaign(product_name: str, budget: float) -> str:
    """Initiates a marketing campaign for a given product within the specified budget."""
    return f"Marketing campaign for {product_name} has been initiated with a budget of ${budget:.2f}."


marketting_head = AssistantAgent(
    name="MarketingAgent",
    description="Marketing Head.",
    system_message="You are a expert marketing agent who come up with ideas to sell products effectively.",
    model_client=gemini_client,
    tools=[initiate_marketing_campaign]
)
chief_data_scientist = AssistantAgent(
    name="DataScientistAgent",
    description="Chief Data Scientist.",
        system_message="You are able to come up with strategies to Analyse existing Data.",
    model_client=gemini_client
)
engineering_head = AssistantAgent(
    name="EngineeringAgent",
    description="CTO.",
    system_message="You are able to come up with new Ideas and come up with Engineering solutions to it.",
    model_client=gemini_client,
    tools=[initiate_marketing_campaign]
)
team = RoundRobinGroupChat(
    participants=[engineering_head, chief_data_scientist, marketting_head],
    max_turns=3
)
final_message = None
# or await team.run and then run loop.
async for message in team.run_stream(task="Come up with an shipping product idea for shipping related project management"):
    final_message = message
    print(f"{'🚀' * 80}\n({type(message)}")
    print(f"type(message) == TaskResult: {type(message) == TaskResult} \n isinstance(message, TaskResult) {isinstance(message, TaskResult)}")
    if type(message) == TextMessage:
        print(f"{'.' * 80}\n[{message.source}] {message.content}")
    else: # Task result has no message source or content.
        print(message)

# Cheeky function inside function
async def async_print_stop_reason(message):
    print("🔨" * 80)
    print(message.stop_reason)
await async_print_stop_reason(final_message)

🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
(<class 'autogen_agentchat.messages.TextMessage'>
type(message) == TaskResult: False 
 isinstance(message, TaskResult) False
................................................................................
[user] Come up with an shipping product idea for shipping related project management
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
(<class 'autogen_agentchat.messages.TextMessage'>
type(message) == TaskResult: False 
 isinstance(message, TaskResult) False
................................................................................
[EngineeringAgent] Here's a shipping product idea focused on project management, along with an engineering solution outline:

## Product Idea: LogiFlow - Intelligent Shipping Project Management Suite

**Concept:** LogiFlow is an AI-powered, collaborative platform designed to centralize, automate, and optimize the end-to-end project managem

In [76]:
def initiate_marketing_campaign(product_name: str, budget: float) -> str:
    """Initiates a marketing campaign for a given product within the specified budget."""
    return f"Marketing campaign for {product_name} has been initiated with a budget of ${budget:.2f}."


marketting_head = AssistantAgent(
    name="MarketingAgent",
    description="Marketing Head.",
    system_message="You are a expert marketing agent who come up with ideas to sell products effectively.",
    model_client=gemini_client,
    tools=[initiate_marketing_campaign]
)
chief_data_scientist = AssistantAgent(
    name="DataScientistAgent",
    description="Chief Data Scientist.",
        system_message="You are able to come up with strategies to Analyse existing Data.",
    model_client=gemini_client
)
engineering_head = AssistantAgent(
    name="EngineeringAgent",
    description="CTO.",
    system_message="You are able to come up with new Ideas and come up with Engineering solutions to it.",
    model_client=gemini_client,
    tools=[initiate_marketing_campaign]
)
team = Swarm(
    participants=[engineering_head, chief_data_scientist, marketting_head],
    max_turns=3
)
result = await team.run(task="Come up with an shipping product idea for shipping related project management")
for message in result.messages:
    print(f"{'*' * 80}\n[{message.source}]: {message.content}")


********************************************************************************
[user]: Come up with an shipping product idea for shipping related project management
********************************************************************************
[EngineeringAgent]: Here's a shipping product idea:

**Product Name: ShipFlow PM**

**Concept:** ShipFlow PM is an AI-powered project management platform specifically designed for the complexities of the shipping and logistics industry. It aims to streamline operations, reduce delays, and improve communication across all stakeholders in a shipping project, from planning to final delivery.

**Key Features:**

1.  **Intelligent Route Optimization & Scheduling:**
    *   **AI-driven analysis:** Leverages real-time data (weather, traffic, port congestion, carrier availability) to suggest the most efficient and cost-effective routes for multi-modal shipments.
    *   **Dynamic scheduling:** Automatically adjusts timelines and alerts stakeholders t